In [1]:
from subprocess import check_output
import subprocess
import os
import sys
import json
import networkx as nx
import matplotlib.pyplot as plt 
from pyvis.network import Network
from Modules.NetVisualizer.net_visualizer import *
from Modules.NetBuilder.net_builder import net_build
from Utils.scripts import visualize_ratas_json
import pandas as pd
from random import randint

# DaScra
Uses DaScra code to mine the archive and get a set of stories in a fandom and their information

In [ ]:
# story_set_metadata=['node', './Modules/DaScra/scraper.js']
# d = check_output(story_set_metadata)

data_to_read='./OutputFiles/dascra_output.xlsx'

stories_df= pd.read_excel(data_to_read,
              dtype={
                  'Additional Tags:': str, 'Archive Warning:': str,
                  'Author:': str, 'Bookmarks:': str,
                  'Category:': str, 'Chapters:': str,
                  'Characters:': str, 'Comments:': str,
                  'Fandom:': str, 'Hits:': int,
                  'Kudos:': int, 'Language:': str,
                  'Rating:': str, 'Relationship:': str,
                  'Series:': str, 'Part:': float,
                  'Source URL:': str, 'Title:': str,
                  'Updated:': str, 'Words:': str,
              }) 




stories_df['Additional Tags:']=stories_df['Additional Tags:'].map(str)
# DO it better all in one pass
additional_tags=','.join([','.join(tags.split(",")) for tags in stories_df["Additional Tags:"][:50]]).replace("/",'*s*').replace(".",'*d*').split(',')
print(len(additional_tags))
additional_tags = [item.strip() for item in additional_tags]
additional_tags=list(set(additional_tags))
print(len(additional_tags))

## Cleaning

In [ ]:
stories_df['Additional Tags:']=stories_df['Additional Tags:'].str.split(',')
stories_df['Additional Tags:']=stories_df['Additional Tags:'].apply(lambda x: [item.strip() for item in x])
# stories_df['Additional Tags:']=stories_df['Additional Tags:'].apply(lambda x: ','.join(x))

In [ ]:
stories_df['Title:']=stories_df['Title:'].apply(lambda x: x.strip())
# stories_df['Title:']


# TagScra
From a set of tags it scrap the RATAS in the archive and create JSON files with the info

In [10]:
#TODO add the output file name of the JSON tag-data as parameter
args_to_tags_scraper=['node', './Modules/TagScraping/scraper.js' ]


tag_structure_file_name='current_tag_net'
# additional_tags=["Hogwarts Eighth Year",'Hogwarts Era',"Hogwarts Fourth Year"]
# additional_tags=["Deaf Character", "Disability","Hogwarts Eighth Year",'Hogwarts Era',"Hogwarts Fourth Year"]


# tag_structure_file_name='Disability_current_tag_net'
# additional_tags=["Disability"]


# tag_structure_file_name='Disability_old_tag_net'
# additional_tags=["Disability"]
# args_to_tags_scraper.extend(['-av','old'])



args_to_tags_scraper.extend(['-of',tag_structure_file_name])
args_to_tags_scraper.extend(['-t'])
args_to_tags_scraper.extend(additional_tags)

# p = check_output(args_to_tags_scraper)

with open('test.log', 'wb') as f: 
    process = subprocess.Popen(args_to_tags_scraper, stdout=subprocess.PIPE)
    for c in iter(lambda: process.stdout.read(1), b''): 
        sys.stdout.write(c.decode())



================= 1/495 =================
================= MINING TAG: Unrequited Love =================
PAGE LOG: Type of TAG
PAGE LOG: canonical_tag Tag
================= Waiting 3min =================
================= 2/495 =================
================= MINING TAG: still hunters =================
PAGE LOG: Type of TAG
PAGE LOG: SYNNED Tag
================= Waiting 3min =================
================= 3/495 =================
================= MINING TAG: Crossover =================
PAGE LOG: Type of TAG
PAGE LOG: canonical_tag Tag
================= Waiting 3min =================
================= 4/495 =================
================= MINING TAG: Coffee Shops =================
PAGE LOG: Type of TAG
PAGE LOG: canonical_tag Tag
================= Waiting 3min =================
================= 5/495 =================
================= MINING TAG: Protective Dean =================
PAGE LOG: Type of TAG
PAGE LOG: SYNNED Tag
================= Waiting 3min =================


# Building the Network and Visuals

## More tags

In [3]:

database_path='./OutputFiles/disability_rata_database'

for filename in os.listdir(database_path):
   file_path=os.path.join(os.path.abspath(database_path), filename)
   version=filename.split('_')[2].replace('.json','')
   visualize_ratas_json(file_path,headings='Disability Rata '+version, file_name='disability_rata_'+version)


In [ ]:
database_path='./OutputFiles/ableism_rata_database'

for filename in os.listdir(database_path):
   file_path=os.path.join(os.path.abspath(database_path), filename)
   version=filename.split('_')[2].replace('.json','')
   visualize_ratas_json(file_path,headings='Ableism Rata '+version, file_name='ableism_rata_'+version)

In [2]:
visualize_ratas_json('./OutputFiles/Disability_old_tag_net.json',headings='Disability old', file_name='disability_old_testing')


(<networkx.classes.digraph.DiGraph at 0x224ed19c148>,
 <class 'pyvis.network.Network'> |N|=250 |E|=265)

In [12]:
with open('./OutputFiles/'+tag_structure_file_name+'.json', 'r', encoding='utf-8') as f:
  current_data = json.load(f)
f.close()

G = net_build(current_data)

current_dis=net_visualize(G, hierarchical_layout=False, node_sizes=True,headings="Current Tag Network:", file_name=tag_structure_file_name)

In [21]:
args_to_tags_scraper=['node', './Modules/TagScraping/scraper.js' ]

args_to_tags_scraper.extend(['-of',"old_tag_net"])
args_to_tags_scraper.extend(['-t'])
args_to_tags_scraper.extend(additional_tags[:190])
args_to_tags_scraper.extend(['-av', 'old'])

# p = check_output(args_to_tags_scraper)

with open('test.log', 'wb') as f: 
    process = subprocess.Popen(args_to_tags_scraper, stdout=subprocess.PIPE)
    for c in iter(lambda: process.stdout.read(1), b''): 
        sys.stdout.write(c.decode())


================= 1/190 =================
================= MINING TAG: Unrequited Love =================
PAGE LOG: $donato = %o JSHandle@node
PAGE LOG: installed message listener to #donato-base: %o JSHandle@node
PAGE LOG: Type of TAG
PAGE LOG: canonical_tag Tag
================= Waiting 3min =================
================= 2/190 =================
================= MINING TAG: still hunters =================
PAGE LOG: $donato = %o JSHandle@node
PAGE LOG: installed message listener to #donato-base: %o JSHandle@node
PAGE LOG: Type of TAG
PAGE LOG: FreeForm Tag
================= Waiting 3min =================
================= 3/190 =================
================= MINING TAG: Crossover =================
PAGE LOG: $donato = %o JSHandle@node
PAGE LOG: installed message listener to #donato-base: %o JSHandle@node
PAGE LOG: Type of TAG
PAGE LOG: canonical_tag Tag
================= Waiting 3min =================
================= 4/190 =================
================= MINING TAG: Co

In [23]:
with open('./OutputFiles/Disability_current_tag_net.json', 'r', encoding='utf-8') as f:
  current_data = json.load(f)
f.close()

H = net_build(current_data)

# current_dis=net_visualize(H, hierarchical_layout=False, node_sizes=True,headings="Old Tag Network:"+','.join(additional_tags), file_name=tag_structure_file_name)

In [25]:
j= G.copy()

j.add_nodes_from(H.nodes)
j.add_edges_from(H.edges)

for node in j.nodes:
    j.nodes[node]['color']= '#19A968' if H.has_node(node) else '#E67E22'

for x,y in j.edges:
    j.edges[x,y]['color']= '#19A968' if H.has_edge(x,y) else '#E67E22'    

In [26]:
current_dis=net_visualize(j, hierarchical_layout=False, node_sizes=True,headings="Combination of different tags in Supernatural y Disability-Tag_network", file_name='combo_supernatural_disability')


In [27]:
j= G.copy()

for node in j.nodes:
    j.nodes[node]['color']= '#19A968' if H.has_node(node) else '#E67E22'

for x,y in j.edges:
    j.edges[x,y]['color']= '#19A968' if H.has_edge(x,y) else '#E67E22'  

In [ ]:
current_dis=net_visualize(j, hierarchical_layout=False, node_sizes=True,headings="aaa", file_name='prueba')


## Current Disability Tag Network

Opens the already mined Disability Tag network
Builds the network
Visualizes it

In [ ]:
tag_structure_file_name='Disability_current_tag_net'
additional_tags=["Disability"]

with open('./OutputFiles/'+tag_structure_file_name+'.json', 'r', encoding='utf-8') as f:
  current_data = json.load(f)
f.close()

G = net_build(current_data)

current_dis=net_visualize(G, hierarchical_layout=False, node_sizes=True,headings="Disability Current Tag Network", file_name=tag_structure_file_name)


## OLD Disability Tag Network

Opens the already mined Disability Tag network
Builds the network
Visualizes it

In [ ]:
tag_structure_file_name='Disability_old_tag_net'
additional_tags=["Disability"]

with open('./OutputFiles/'+tag_structure_file_name+'.json', 'r', encoding='utf-8') as f:
  old_data = json.load(f)
f.close()

H = net_build(old_data)

# old_dis=net_visualize(H, hierarchical_layout=True, node_sizes=True,headings="Disability Old Tag Network", file_name=tag_structure_file_name)
old_dis=net_visualize(H, hierarchical_layout=False, node_sizes=True,headings="Disability Old Tag Network", file_name=tag_structure_file_name)
# net_visualize(H, hierarchical_layout=False, node_sizes=True,headings="Tag Network of: "+ ",".join(additional_tags), file_name=tag_structure_file_name)


## Similarities with Color-Layout


In [ ]:
new_vis, old_vis = get_visuals_older_vs_newest(H.copy(), G.copy(),hierarchical_layout=False)
new_vis.show("Disability_current_tag_net_updated.html")
old_vis.show("Disability_old_tag_net_updated.html")

In [ ]:
new_vis, old_vis = get_visuals_older_vs_newest(H.copy(), G.copy(),hierarchical_layout=False)
new_vis.show("current_tag_net_updated.html")
old_vis.show("old_tag_net_updated.html")

## Getting the difference


### Current Disability Tag-Net - Old Disability Tag-Net

Red represents the nodes and edges that will be removed in the difference, the ones that belong to the current verion that also belon to the older

#### Blue represents what it is in the current version that was not in the older version == NEW

In [ ]:
diff_G_H,diff_G_H_vis=get_vis_G_diff_H(G,H,False,title="Diff new-old Disability")
diff_G_H_vis.show("diff_Disability_new_old.html")

### Old Disability Tag-Net - Current Disability Tag-Net

Red represents the nodes and edges that will be removed in the difference, the ones that belong to the old verion that also belong to the current

#### Blue represents what it was in the Old version that is not there in the current version == CHANGES, EVOLUTION

In [ ]:
diff_H_G,diff_H_G_vis=get_vis_G_diff_H(H,G,False, title="Diff old-new Disability")
diff_H_G_vis.show("diff_Disability_old_new.html")

In [ ]:
diff_H_G,diff_H_G_vis=get_vis_G_diff_H(H,G,False, title="Diff old-new")
diff_H_G_vis.show("diff_old_new.html")

# Connecting Stories using the RATAS

In [ ]:
S=nx.Graph()
S.add_nodes_from(stories_df['Title:'])
for story in stories_df['Title:']:
    for conection in stories_df.loc[stories_df['Additional Tags:'].apply(lambda x: 'Fluff' in set(x)), 'Title:']:
        S.add_edge(story,conection) if not story==conection else None

net_visualize(S,False,False,"Full Story-Tag Net",'full_story_net')

In [ ]:
S=nx.Graph()
S.add_nodes_from(stories_df['Title:'])
for story in stories_df['Title:']:
    for conection in stories_df.loc[stories_df['Additional Tags:'].apply(lambda x: 'Fluff' in set(x)), 'Title:']:
        S.add_edge(story,conection) if not story==conection else None
        # S[story][conection]['color']='#2196F3' if not edge in list_edges else '#E91E63'

net_visualize(S,False,False,"Full Story-Tag Net",'full_story_net')